### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`. Additionally, there are several countries with Numeric digits in their name. Make sure to remove these as well, e.g. `'Italy9'` should be `'Italy'`. 

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [2]:
import pandas as pd 
import numpy as np
import re

In [3]:
Energy=pd.read_excel("assets/Energy Indicators.xls")

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

In [4]:
pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 704.9 kB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=16)

In [3]:
Energy.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Energy Supply,Energy Supply per capita,Renewable Electricity Production
0,NaN,NaN,NaN,Petajoules,Gigajoules,%
1,NaN,Afghanistan,Afghanistan,321,10,78.66928
2,NaN,Albania,Albania,102,35,100
3,NaN,Algeria,Algeria,1959,51,0.55101
4,NaN,American Samoa,American Samoa,...,...,0.641026


In [33]:
len(Energy)

228

In [34]:
Energy.loc[227]

Unnamed: 0                               NaN
Unnamed: 1                          Zimbabwe
Unnamed: 2                          Zimbabwe
Energy Supply                            480
Energy Supply per capita                  32
Renewable Electricity Production    52.53612
Name: 227, dtype: object

In [35]:
Energy.iloc[0]

Unnamed: 0                                 NaN
Unnamed: 1                                 NaN
Unnamed: 2                                 NaN
Energy Supply                       Petajoules
Energy Supply per capita            Gigajoules
Renewable Electricity Production             %
Name: 0, dtype: object

In [45]:
Energy=Energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)

In [58]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=17)
Energy=Energy.drop([Energy.columns[0],Energy.columns[1]],axis=1)
Energy=Energy.rename(columns={'Unnamed: 2':'Country','Petajoules':'Energy Supply','Gigajoules':'Energy Supply per Capita','%':'% Renewable'})
Energy.replace("...",np.nan,inplace=True)
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']
Energy[Energy.columns[0]]=Energy[Energy.columns[0]].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States", 
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
})

Energy=Energy.set_index('Country')
Energy.rename(index=lambda x: x.replace(r'\([^)]*\)', '').strip(), inplace=True)

countries= Energy.index.tolist()
countries = [re.sub(r'\([^)]*\)', '', country).strip() for country in countries]
countries = [re.sub(r'\d+', '', country).strip() for country in countries]
Energy.index=countries

GDP=pd.read_csv('assets/world_bank.csv',skiprows=4)

GDP[GDP.columns[0]]=GDP[GDP.columns[0]].replace({
"Korea, Rep.": "South Korea",  "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"
})

GDP=GDP.set_index(GDP.columns[0])
countries= GDP.index.tolist()

countries = [re.sub(r'\([^)]*\)', '', country).strip() for country in countries]
countries = [re.sub(r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*\s\(\w[^()]*\)', '', country).strip() for country in countries]
GDP.index=countries

ScimEn=pd.read_excel('assets/scimagojr-3.xlsx')
ScimEn=ScimEn.set_index(ScimEn.columns[1])

data=pd.merge(pd.merge(ScimEn.iloc[:15,:],Energy,left_index=True, right_index=True, how='left'), GDP.iloc[:,-10:] ,left_index=True, right_index=True, how='left')
data=data.sort_values(by='Rank', ascending=True)
print(data.shape)
data.head()




In [48]:
data

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,NaN,NaN,NaN,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,NaN,NaN,NaN,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [57]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=17)
Energy.columns[0]

'Unnamed: 0'

In [53]:
Energy.index.to_list()

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire, Sint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'China, Hong Kong Special Administrative Region',
 'China, Macao Special Administrative Region',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Côte d'Ivoire",
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 "Democratic People's Republic of Korea",
 'Democratic Republic of the Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 

In [35]:
pip install --upgrade openpyxl 'Korea, Rep.' 'Korea, Dem. People’s Rep.'

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 1.9 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
GDP.iloc[:, :3].head()

,Country Code,Indicator Name,Indicator Code
Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD
Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD
Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD
Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD
Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD


In [50]:
Energy=Energy.set_index('Country')

In [51]:
Energy.head()

,Energy Supply,Energy Supply per capita,% Renewable
Country,,,
Afghanistan,321,10,78.66928
Albania,102,35,100
Algeria,1959,51,0.55101
American Samoa,...,...,0.641026
Andorra,9,121,88.69565


In [22]:
GDP.iloc[:, -10:-1].head()

,2006,2007,2008,2009,2010,2011,2012,2013,2014
Aruba,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN
Andorra,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN
Afghanistan,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10
Angola,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11
Albania,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10


In [52]:
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']

In [12]:
ScimEn=pd.read_excel('assets/scimagojr-3.xlsx')
ScimEn
ScimEn.head()


,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57


In [54]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=16)
Energy=Energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
Energy=Energy.rename(columns={'Unnamed: 2':'Country','Renewable Electricity Production':'% Renewable'})
Energy=Energy.drop(0)
Energy=Energy.set_index('Country')
Energy.replace("...",np.nan,inplace=True)
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']
Energy.head()

,Energy Supply,Energy Supply per capita,% Renewable
Country,,,
Afghanistan,3.210000e+08,10.0,78.66928
Albania,1.020000e+08,35.0,100
Algeria,1.959000e+09,51.0,0.55101
American Samoa,NaN,NaN,0.641026
Andorra,9.000000e+06,121.0,88.69565


In [71]:

Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=16)
Energy=Energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
Energy=Energy.rename(columns={'Unnamed: 2':'Country','Renewable Electricity Production':'% Renewable'})
Energy=Energy.drop(0)
Energy.replace("...",np.nan,inplace=True)
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']
Energy['Country']=Energy['Country'].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
})
Energy.head()




,Country,Energy Supply,Energy Supply per capita,% Renewable
1,Afghanistan,3.210000e+08,10.0,78.669280
2,Albania,1.020000e+08,35.0,100.000000
3,Algeria,1.959000e+09,51.0,0.551010
4,American Samoa,NaN,NaN,0.641026
5,Andorra,9.000000e+06,121.0,88.695650


In [72]:
Energy['Country']=Energy['Country'].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
})

In [77]:
Energy=Energy.set_index('Country')


In [87]:
Energy.loc['South Korea']

Energy Supply               1.100700e+10
Energy Supply per capita    2.210000e+02
% Renewable                 2.279353e+00
Name: South Korea, dtype: float64

In [81]:
Energy.head()

,Energy Supply,Energy Supply per capita,% Renewable
Country,,,
Afghanistan,3.210000e+08,10.0,78.669280
Albania,1.020000e+08,35.0,100.000000
Algeria,1.959000e+09,51.0,0.551010
American Samoa,NaN,NaN,0.641026
Andorra,9.000000e+06,121.0,88.695650


In [89]:
print(Energy.index.tolist())

['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia1', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China2', 'China, Hong Kong Special Administrative Region3', 'China, Macao Special Administrative Region4', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark5', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador'

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=16)
Energy=Energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
Energy=Energy.rename(columns={'Unnamed: 2':'Country','Renewable Electricity Production':'% Renewable'})
Energy=Energy.drop(0)
Energy.replace("...",np.nan,inplace=True)
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']
Energy.head()

Energy['Country']=Energy['Country'].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
})

Energy=Energy.set_index('Country')
Energy.rename(index=lambda x: x.replace(r'\([^)]*\)', '').strip(), inplace=True)

In [3]:
print(Energy.index.tolist())

['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia1', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia (Plurinational State of)', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China2', 'China, Hong Kong Special Administrative Region3', 'China, Macao Special Administrative Region4', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark5', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador'

In [16]:
countries= Energy.index.tolist()

In [17]:
countries = [re.sub(r'\([^)]*\)', '', country).strip() for country in countries]

In [7]:
import re

In [9]:
print(countries)


['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia1', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China2', 'China, Hong Kong Special Administrative Region3', 'China, Macao Special Administrative Region4', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark5', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',

In [18]:
countries = [re.sub(r'\d+', '', country).strip() for country in countries]

In [19]:
print(countries)

['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'China, Hong Kong Special Administrative Region', 'China, Macao Special Administrative Region', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', "Democratic People's Republic of Korea", 'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equ

In [ ]:
Energy.index=countries

In [20]:
Energy=pd.read_excel("assets/Energy Indicators.xls", skipfooter=38,skiprows=16)
Energy=Energy.drop(['Unnamed: 0','Unnamed: 1'],axis=1)
Energy=Energy.rename(columns={'Unnamed: 2':'Country','Renewable Electricity Production':'% Renewable'})
Energy=Energy.drop(0)
Energy.replace("...",np.nan,inplace=True)
Energy['Energy Supply']=(10**6)*Energy['Energy Supply']
Energy.head()

Energy['Country']=Energy['Country'].replace({
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
})

Energy=Energy.set_index('Country')
Energy.rename(index=lambda x: x.replace(r'\([^)]*\)', '').strip(), inplace=True)

countries = [re.sub(r'\([^)]*\)', '', country).strip() for country in countries]
countries = [re.sub(r'\d+', '', country).strip() for country in countries]
Energy.index=countries

In [22]:
Energy.loc['Italy']

Energy Supply               6.530000e+09
Energy Supply per capita    1.090000e+02
% Renewable                 3.366723e+01
Name: Italy, dtype: float64

In [29]:
df1 = pd.DataFrame({
    'A': ['A1', 'A2', 'A3']
}, index=[0, 1, 2])

df2 = pd.DataFrame({
    'B': ['B1', 'B2', 'B3', 'B4']
}, index=[0, 1, 2, 3])

df3 = pd.DataFrame({
    'C': ['C1', 'C2', 'C3', 'C4']
}, index=[1, 2, 3, 4])

# First merge df1 and df2 on their index, keeping only the data in df1
merged_df = pd.merge(df1, df2, left_index=True, right_index=True, how='left')

# Then merge the result with df3 on their index, still keeping only the data in df1
merged_df = pd.merge(merged_df, df3, left_index=True, right_index=True, how='left')

# Display the merged DataFrame
print(merged_df)

    A   B    C
0  A1  B1  NaN
1  A2  B2   C1
2  A3  B3   C2
